In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from scipy.sparse import csr_matrix

from implicit.als import AlternatingLeastSquares
from implicit.nearest_neighbours import bm25_weight, tfidf_weight

from IPython.display import clear_output
from tqdm import tqdm
tqdm.pandas()

import os, sys

module_path = os.path.abspath(os.path.join(os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

from metrics import precision_at_k

In [2]:
os.environ['MKL_NUM_THREADS'] = '1'

#### Загружаем данные и предобрабатываем их:

In [3]:
data = pd.read_csv('data/transaction_data.csv')

data.columns = [col.lower() for col in data.columns]
data.rename(columns={'household_key': 'user_id',
                     'product_id': 'item_id'},
            inplace=True)

test_size_weeks = 3

data_train = data[data['week_no'] < data['week_no'].max() - test_size_weeks]
data_test = data[data['week_no'] >= data['week_no'].max() - test_size_weeks]
data_train.head(4)

,user_id,basket_id,day,item_id,quantity,sales_value,store_id,retail_disc,trans_time,week_no,coupon_disc,coupon_match_disc
0,2375,26984851472,1,1004906,1,1.39,364,-0.6,1631,1,0.0,0.0
1,2375,26984851472,1,1033142,1,0.82,364,0.0,1631,1,0.0,0.0
2,2375,26984851472,1,1036325,1,0.99,364,-0.3,1631,1,0.0,0.0
3,2375,26984851472,1,1082185,1,1.21,364,0.0,1631,1,0.0,0.0


In [4]:
item_features = pd.read_csv('data/product.csv')
item_features.columns = [col.lower() for col in item_features.columns]
item_features.rename(columns={'product_id': 'item_id'}, inplace=True)
item_features.head(4)

,item_id,manufacturer,department,brand,commodity_desc,sub_commodity_desc,curr_size_of_product
0,25671,2,GROCERY,National,FRZN ICE,ICE - CRUSHED/CUBED,22 LB
1,26081,2,MISC. TRANS.,National,NO COMMODITY DESCRIPTION,NO SUBCOMMODITY DESCRIPTION,
2,26093,69,PASTRY,Private,BREAD,BREAD:ITALIAN/FRENCH,
3,26190,69,GROCERY,Private,FRUIT - SHELF STABLE,APPLE SAUCE,50 OZ


In [5]:
result = data_test.groupby('user_id')['item_id'].unique().reset_index()
result.columns=['user_id', 'actual']
result.head(4)

,user_id,actual
0,1,"[879517, 934369, 1115576, 1124029, 5572301, 65..."
1,3,"[823704, 834117, 840244, 913785, 917816, 93870..."
2,5,"[913077, 1118028, 1386668]"
3,6,"[825541, 859676, 999318, 1055646, 1067606, 108..."


In [6]:
popularity = data_train.groupby('item_id')['quantity'].sum().reset_index()
popularity.rename(columns={'quantity': 'n_sold'}, inplace=True)

top_5000 = popularity.sort_values('n_sold', ascending=False).head(5000).item_id.tolist()

In [7]:
data_train.loc[~data_train['item_id'].isin(top_5000), 'item_id'] = 999_999

#### Создадим функцию, при помощи которой можно будет применять ALS с настройкой весов и параметров:

In [8]:
def als_recommendation(values='quantity', aggfunc='count', factors=100, regularization=0.001): 
    user_item_matrix = pd.pivot_table(data_train, 
                                      index='user_id',
                                      columns='item_id',
                                      values= values,
                                      aggfunc=aggfunc,
                                      fill_value=0
                                     )

    user_item_matrix = user_item_matrix.astype(float)

    sparse_user_item = csr_matrix(user_item_matrix).tocsr()

    userids = user_item_matrix.index.values
    itemids = user_item_matrix.columns.values

    matrix_userids = np.arange(len(userids))
    matrix_itemids = np.arange(len(itemids))

    id_to_itemid = dict(zip(matrix_itemids, itemids))
    id_to_userid = dict(zip(matrix_userids, userids))

    itemid_to_id = dict(zip(itemids, matrix_itemids))
    userid_to_id = dict(zip(userids, matrix_userids))

    model = AlternatingLeastSquares(factors=factors,
                                    regularization=regularization,
                                    iterations=15,
                                    calculate_training_loss=True,
                                    num_threads=4)

    model.fit(csr_matrix(user_item_matrix).T.tocsr(),
              show_progress=True)


    def get_recommendations(user, model, n=5):
        res = [id_to_itemid[rec[0]] for rec in 
               model.recommend(userid=userid_to_id[user],
                               user_items=sparse_user_item,
                               N=n,
                               filter_already_liked_items=False,
                               filter_items=None,
                               recalculate_user=True
                              )]

        return res


    result['als'] = result['user_id'].progress_apply(lambda x: get_recommendations(x, model=model, n=5))
    precision_res = result.progress_apply(lambda row: precision_at_k(row['als'], row['actual']), axis=1).mean()
    clear_output(wait=True)

    return precision_res

#### Возьмём следующие признаки: quantity, sales_value, store_id, а также посчитаем по каждому sum, count, mean и median:

In [9]:
%%time

values = ['quantity', 'sales_value', 'store_id']
aggfunc_list = ['sum', 'count', 'mean', 'median']
results = {'values': [], 'aggfunc': [], 'precision': []} 

for value in values:
    for aggfunc in aggfunc_list:
        results['values'].append(value)
        results['aggfunc'].append(aggfunc)
        results['precision'].append(als_recommendation(values=value, aggfunc=aggfunc))

pd.DataFrame(results)

Wall time: 8min 12s


,values,aggfunc,precision
0,quantity,sum,0.131391
1,quantity,count,0.147664
2,quantity,mean,0.156605
3,quantity,median,0.157911
4,sales_value,sum,0.090909
5,sales_value,count,0.148569
6,sales_value,mean,0.127574
7,sales_value,median,0.129483
8,store_id,sum,0.007333
9,store_id,count,0.148167


#### Видим, что признак quantity и подсчёт по нему медианы показывает лучшие результаты для ALS.

#### Попробуем определить оптимальное количество факторов и лямбду для quantity + median:

In [10]:
%%time

factors = [10, 25, 50, 75, 100, 150, 200]
regularization_list = [0.1, 0.01, 0.001, 0.0001]
results = {'factors': [], 'lambda': [], 'precision': []} 

for factor in factors:
    for regularization in regularization_list:
        results['factors'].append(factor)
        results['lambda'].append(regularization)
        results['precision'].append(als_recommendation(values='quantity',
                                                       aggfunc='median',
                                                       factors=factor,
                                                       regularization=regularization
                                                      ))

results_df = pd.DataFrame(results)
results_df

Wall time: 14min 3s


,factors,lambda,precision
0,10,0.1000,0.156806
1,10,0.0100,0.157710
2,10,0.0010,0.161326
3,10,0.0001,0.160020
4,25,0.1000,0.161929
5,25,0.0100,0.157710
6,25,0.0010,0.158915
7,25,0.0001,0.159116
8,50,0.1000,0.160824
9,50,0.0100,0.158413


#### Видим, что, в основном, самый высокий результат при различном количестве факторов наблюдается при lambda = 0.1 (кроме 10 факторов, где теряется много информации). Выведем отдельную таблицу с лямбдой равной 0.1:

In [11]:
results_df.loc[(results_df['lambda']==0.1)]

,factors,lambda,precision
0,10,0.1,0.156806
4,25,0.1,0.161929
8,50,0.1,0.160824
12,75,0.1,0.161728
16,100,0.1,0.161627
20,150,0.1,0.160723
24,200,0.1,0.155399


#### Видим, что наибольшая точность наблюдается при 25 факторах. Так как у нас сильно разреженные матрицы, то большое количество факторов и, соответственно, меньший процент потери информации, не дают гарантий улучшения качества и повышения точности. Поэтому мы можем потерять некую часть информации, которая не имеет особого влияния, после чего предсказывать только на хороших значениях (при большем количестве факторов пришлось бы предсказывать на значительном количестве нулевых значений, из-за чего бы снизилась точность).